## Notebook 03 — Render Report

Two options:

Python call to nbconvert/pandoc to knit templates/ClinPharm_Summary_template.md → PDF.

In [9]:
# Pure-Python render: Jinja2 -> Markdown -> HTML (no Quarto/R)
# Fix: template string is a PLAIN triple-quoted string (NOT an f-string).

from pathlib import Path
import yaml, datetime
from jinja2 import Template

try:
    import markdown as md
except ImportError:
    raise SystemExit("Please install python-markdown: pip install markdown")

# --- Load config and set paths ---
cfg = yaml.safe_load(open("config.yaml", "r"))
OUTD = Path(cfg["output"]["dir"]); OUTD.mkdir(parents=True, exist_ok=True)
FIGD = Path(cfg["output"]["figures"])

params = {
    "title":   cfg["report"]["title"],
    "sponsor": cfg["report"]["sponsor"],
    "product": cfg["report"]["product"],
    "required_tol": cfg["report"]["required_tolerance_ugmL"],
    "date": datetime.date.today().isoformat(),
    "fig": {
        "tolerance": str(FIGD / "recovery_overlay_sim_vs_lit.png"),
        "passalert": str(FIGD / "pass_alert_table.png"),
        "wk52box":   str(FIGD / "wk52_by_detected_ADA_assay.png"),
        "adaexp":    str(FIGD / "ada_detection_vs_exposure.png"),
        "er_true":   str(FIGD / "exposure_response_true.png"),
    },
    "tables_dir": str(OUTD),
}

# --- PLAIN (non-f) template string so {{ ... }} survives for Jinja2 ---
template_md = """
# {{ title }}

**Product:** {{ product }} • **Sponsor:** {{ sponsor }} • **Date:** {{ date }}

## 1. Bioanalytical Methods & Interference
PandA (PEG + Acid) improves drug tolerance vs standard bridging (≥80% recovery up to ≥{{ required_tol }} µg/mL).

![Drug tolerance]({{ fig.tolerance }})

**PASS/ALERT summary**

![PASS/ALERT table]({{ fig.passalert }})

## 2. Population PK & Variability (brief)
At Week 52, ADA⁺ subjects show reduced troughs.

![Week-52 troughs by detected ADA]({{ fig.wk52box }})

## 3. Exposure–Response & Target Attainment
True exposure–response shows reduced efficacy at low troughs.

![True exposure–response]({{ fig.er_true }})

Target attainment table (CSV): `{{ tables_dir }}/TLG_target_attainment.csv`

## 4. Dose Recommendation (draft)
Use **PandA-based immunogenicity** for PK/E–R decisions at therapeutic drug levels.
If program requires ≥X% target attainment, consider dose adjustment **only if** safety margin allows.

## 5. Conclusions
- Standard bridging fails tolerance benchmark; PandA passes.
- Bioanalytical method selection materially changes PK/E–R interpretation and dose justification.
"""

# --- Render Markdown with Jinja2 ---
md_text = Template(template_md).render(**params)

# --- Save Markdown ---
md_path = OUTD / "ClinPharm_Summary.md"
md_path.write_text(md_text, encoding="utf-8")
print("Wrote:", md_path)

# --- Convert to standalone HTML (python-markdown) ---
css = """
<style>
body { font-family: -apple-system, Segoe UI, Roboto, Helvetica, Arial, sans-serif; line-height: 1.45; max-width: 860px; margin: 2rem auto; padding: 0 1rem; }
h1, h2, h3 { line-height: 1.2; }
img { max-width: 100%; height: auto; border: 1px solid #eee; padding: 4px; background: #fafafa; }
code { background: #f6f8fa; padding: 0.2em 0.4em; border-radius: 4px; }
hr { border: 0; height: 1px; background: #eee; margin: 2rem 0; }
</style>
"""
html_body = md.markdown(md_text, extensions=["tables", "fenced_code"])
html = f"<!DOCTYPE html><html><head><meta charset='utf-8'><title>{params['title']}</title>{css}</head><body>{html_body}</body></html>"

html_path = OUTD / "ClinPharm_Summary.html"
html_path.write_text(html, encoding="utf-8")
print("Wrote:", html_path)
print("Open the HTML and use your browser’s Print → Save as PDF if you need a PDF.")

Wrote: /Users/cmontefusco/Coding projects/regulatory-style-clinpharm-report/output/ClinPharm_Summary.md
Wrote: /Users/cmontefusco/Coding projects/regulatory-style-clinpharm-report/output/ClinPharm_Summary.html
Open the HTML and use your browser’s Print → Save as PDF if you need a PDF.
